In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [2]:
# params
batch_size = 100

# data downloading - MNIST 28x28x1; 10 classes; 60,000 train, 10,000 test
train = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

# data loader
torch.manual_seed(42)
kwargs = {'num_workers': 2, 'pin_memory': True} if device=='cuda' else {}
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False, **kwargs)

In [3]:
# should be 5
print(train_loader.dataset.targets[0])

tensor(5)


In [15]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(28*28*1, 64)
    self.fc2 = nn.Linear(64, 10)
  def forward(self, x):
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    return F.log_softmax(x, dim=1)

def train_model(data_loader, model, num_epochs):
  # initialization
  model.train()
  # optimization
  opt = optim.Adam(model.parameters(), lr=0.01)
  L = nn.CrossEntropyLoss()
  # training
  losses = []
  mean_losses = []
  accuracy = []
  mean_accuracy = []
  for epoch in range(num_epochs):
    epoch_losses = []
    epoch_accuracy = []
    for batch_idx, (data, target) in enumerate(data_loader):
      data = data.to(device)
      target = target.to(device)
      # data = data.view(data.shape[0], -1)
      opt.zero_grad()
      output = model(data)
      loss = L(output, target)
      loss.backward()
      opt.step()
      epoch_losses.append(loss.item())
      epoch_accuracy.append((output.argmax(dim=1)==target).float().mean())
    losses.append(epoch_losses)
    mean_losses.append(torch.tensor(epoch_losses).mean())
    accuracy.append(epoch_accuracy)
    mean_accuracy.append(torch.tensor(epoch_accuracy).mean())
    print(f'Epoch: {epoch+1}, Loss: {mean_losses[epoch]:.6f}, Accuracy: {mean_accuracy[epoch]:.6f}')
  return mean_losses, mean_accuracy

In [16]:
# params
model = Net().to(device)
num_epochs = 5

# train loop
train_model(train_loader, model, num_epochs)

Epoch: 1, Loss: 0.250972, Accuracy: 0.925850
Epoch: 2, Loss: 0.131870, Accuracy: 0.959900
Epoch: 3, Loss: 0.107904, Accuracy: 0.966850
Epoch: 4, Loss: 0.096135, Accuracy: 0.970517
Epoch: 5, Loss: 0.085360, Accuracy: 0.973350


([tensor(0.2510),
  tensor(0.1319),
  tensor(0.1079),
  tensor(0.0961),
  tensor(0.0854)],
 [tensor(0.9259),
  tensor(0.9599),
  tensor(0.9668),
  tensor(0.9705),
  tensor(0.9733)])